This is test code to figure out a minimal implementation using providede faiss functions to crack through python.
This is to be used for the prototype implementation.
The optimized implementation should be native C++ in faiss

In [2]:
import os
import numpy as np
import logging
import faiss
from vasili_helpers import *
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# InteractiveShell.ast_node_interactivity = "last"

%load_ext autoreload
%autoreload 2

In [3]:
xb, xq, xt, gt, metric = load_sift1M(f"/pub/scratch/vmageirakos/datasets/ann-fvecs/sift-128-euclidean")
nb, d = xb.shape

Loading sift1M...

dataset shape:
xb.shape=(1000000, 128)
gt.shape=(10000, 100)
xq.shape=(10000, 128)


done


In [4]:
nlist = 1000
n_iter = 0
max_pts = 256
seed =42 
result_dir= None

In [ ]:
index, train_time, add_time = train_ivfflat(
    xb,
    nlist=nlist,
    km_n_iter=n_iter,
    km_max_pts=max_pts,
    seed=seed,
    store_dir=None,  # if you want to store the index
    verbose=True,
    metric=metric,
)

Kmeans... nlist=1000 km_n_iter=10 km_max_pts=256 seed=42 nredo=1
Training level-1 quantizer
Training level-1 quantizer on 1000000 vectors in 128D
Training IVF residual
IndexIVF: no residual training
IndexIVFFlat::add_core: added 1000000 / 1000000 vectors
	---> Index Train Time = 997.8557150006964 ms | Add Time = 294.3818399999145 ms <---


In [5]:
index.own_invlists # if the index owns the InvertedList object (and can delete it etc. )

True

In [ ]:
dir(index.invlists)

['INVALID_CODE_SIZE',
 'SUBSET_TYPE_ELEMENT_RANGE',
 'SUBSET_TYPE_ID_MOD',
 'SUBSET_TYPE_ID_RANGE',
 'SUBSET_TYPE_INVLIST',
 'SUBSET_TYPE_INVLIST_FRACTION',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__weakref__',
 'add_entries',
 'add_entry',
 'code_size',
 'compute_ntotal',
 'copy_subset_to',
 'get_codes',
 'get_ids',
 'get_iterator',
 'get_single_code',
 'get_single_id',
 'imbalance_factor',
 'is_empty',
 'list_size',
 'merge_from',
 'nlist',
 'prefetch_lists',
 'print_stats',
 'release_codes',
 'release_ids',
 'reset',
 'resize',
 'this',
 'thisown',
 'update_entries',
 'update_entry',
 'use_iterator']

In [7]:
index_to_crack = faiss.clone_index(index) # each run should start from same point because crack is in-place

In [ ]:
index, train_time, add_time = train_ivfflat(
    xb,
    nlist=1,
    km_n_iter=n_iter,
    km_max_pts=max_pts,
    seed=seed,
    store_dir=result_dir,  # if you want to store the index
    verbose=True,
    metric=metric,
)

NameError: name 'train_ivfflat' is not defined

# Skip

What happens if you increase number of centroids and try to replace quantizer?

In [8]:
outer_centroids = index_to_crack.quantizer.reconstruct_n() # or kmeans.centroids idk which is faster
outer_centroids.shape

(1000, 128)

In [9]:
# add 1 more centroid
first_query = xq[0:1, :]
new_centroids = np.vstack([outer_centroids, first_query])

In [10]:
if metric == "euclidean":
    new_quantizer = faiss.IndexFlatL2(d)
elif metric == "angular":
    new_quantizer = faiss.IndexFlatIP(d)
new_quantizer.add(new_centroids)
_ = replace_ivf_quantizer(index_to_crack, new_quantizer) # replace quantizer

AssertionError: 

In [11]:
dir(index_to_crack.invlists)

['INVALID_CODE_SIZE',
 'SUBSET_TYPE_ELEMENT_RANGE',
 'SUBSET_TYPE_ID_MOD',
 'SUBSET_TYPE_ID_RANGE',
 'SUBSET_TYPE_INVLIST',
 'SUBSET_TYPE_INVLIST_FRACTION',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__weakref__',
 'add_entries',
 'add_entry',
 'code_size',
 'compute_ntotal',
 'copy_subset_to',
 'get_codes',
 'get_ids',
 'get_iterator',
 'get_single_code',
 'get_single_id',
 'imbalance_factor',
 'is_empty',
 'list_size',
 'merge_from',
 'nlist',
 'prefetch_lists',
 'print_stats',
 'release_codes',
 'release_ids',
 'reset',
 'resize',
 'this',
 'thisown',
 'update_entries',
 'update_entry',
 'use_iterator']

# Continue

I think Horizontal Slice merging is what we want with InvLists to increase number of inverted lists. At least based on this: https://github.com/facebookresearch/faiss/wiki/Indexing-1T-vectors#split-across-inverted-lists

Try to use combined index:

If you use VStack to stack invlists in this manner, you need to take care not to add query point as point

In [8]:
# new index contains only xq (centroid xq invlists xq etc.)
new_index, train_time, add_time = train_ivfflat(
    xq[0:1,:],
    nlist=1,
    km_n_iter=0,
    km_max_pts=1,
    seed=1,
    store_dir=None,  # if you want to store the index
    verbose=True,
    metric=metric,
)
new_index.remove_ids(np.array([0])) # take care not to have query in the xb of this new index
new_index.ntotal
# new_index.quantizer.reconstruct_n()
# xq[0:1, :]

Kmeans... nlist=1 km_n_iter=0 km_max_pts=1 seed=1 nredo=1
Training level-1 quantizer
Training level-1 quantizer on 1 vectors in 128D
Training IVF residual
IndexIVF: no residual training
IndexIVFFlat::add_core: added 1 / 1 vectors
	---> Index Train Time = 0.4050589996040799 ms | Add Time = 1.154796002083458 ms <---


WARNING clustering 1 points to 1 centroids: please provide at least 39 training points


1

0

In [9]:
indexes = [index_to_crack, new_index]
indexes[0].nlist
indexes[1].nlist

1000

1

In [10]:
ilv = faiss.InvertedListsPtrVector()
for indx in indexes:
    indx.nlist
    il = faiss.extract_index_ivf(indx).invlists
    ilv.push_back(il)
big_il = faiss.VStackInvertedLists(ilv.size(), ilv.data())
# big_il = faiss.HStackInvertedLists(ilv.size(), ilv.data())
big_il.nlist

1000

1

1001

In [11]:
big_il
ntotal_new = big_il.compute_ntotal()
ntotal_new  # take care not to have query in xb of new index
assert ntotal_new == index_to_crack.ntotal

<faiss.swigfaiss_avx2.VStackInvertedLists; proxy of <Swig Object of type 'faiss::VStackInvertedLists *' at 0x7fba9d355920> >

1000000

In [12]:
### SKIP
# big_il # invlists object, we still have to replace old index somehow to have this one?
# index_to_crack.nlist = big_il.nlist # do pass assert in the next one
# index_to_crack.replace_invlists(big_il, False) # TODO : Check that this works as intended...

As you can see quantizer was not replaced, only invlists so this will probably lead to issues

In [14]:
index_to_crack.quantizer.reconstruct_n().shape 
index_to_crack.invlists.nlist


(1000, 128)

1000

In [15]:
new_centroids = np.vstack([index_to_crack.quantizer.reconstruct_n(), new_index.quantizer.reconstruct_n()]) # 
new_centroids.shape

(1001, 128)

### (SKIP) replace quantizer:

In [19]:
# new_centroids = np.vstack([outer_centroids, first_query])
if metric == "euclidean":
    new_quantizer = faiss.IndexFlatL2(d)
elif metric == "angular":
    new_quantizer = faiss.IndexFlatIP(d)
new_quantizer.add(new_centroids)
_ = replace_ivf_quantizer(index_to_crack, new_quantizer) # replace quantizer

Now it seems like it worked, but what about the centroid/index ids what are they now?

In [20]:
index_to_crack.quantizer.reconstruct_n().shape 
index_to_crack.invlists.nlist

(1001, 128)

1001

```
 'add_entries',
 'add_entry',
 'code_size',
 'compute_ntotal',
 'copy_subset_to',
 'get_codes',
 'get_ids',
 'get_iterator',
 'get_single_code',
 'get_single_id',
 'imbalance_factor',
 'is_empty',
 'list_size',
 'merge_from',
 'nlist',
 'prefetch_lists',
 'print_stats',
 'release_codes',
 'release_ids',
 'reset',
 'resize',
 'this',
 'thisown',
 'update_entries',
 'update_entry',
 'use_iterator'
 ```

Doesn't work, kernel crash

In [ ]:
index_to_crack.invlists.print_stats()

: 

### (CONTINUE) What if we create new index with specific quantizer and invlists as we set it? Will it then be correctly initialized?

In [16]:
new_nlist = big_il.nlist
new_nlist
new_centroids = np.vstack([index_to_crack.quantizer.reconstruct_n(), new_index.quantizer.reconstruct_n()]) # 
new_centroids.shape

1001

(1001, 128)

In [33]:
new_quantizer = faiss.IndexFlatL2(d)
new_quantizer
new_quantizer.add(new_centroids)
# new empty index
new_index = faiss.IndexIVFFlat(new_quantizer, d, new_nlist, faiss.METRIC_L2)
new_index.ntotal
new_index.is_trained
new_index.nlist
new_index.invlists.nlist
new_index.invlists.print_stats() # empty

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7fba9d355d40> >

0

True

1001

1001

list size in < 1: 1001 instances


In [34]:
new_index.replace_invlists
big_il # invlists object, we still have to replace old index somehow to have this one?
new_index.nlist = big_il.nlist # do pass assert in the next one
new_index.replace_invlists(big_il, False) # True/False is wether own_invlists ( ie if index is deleted this arrayinvlists is deleted)
new_index.own_invlists
new_index.ntotal
new_index.is_trained
new_index.nlist
new_index.invlists.nlist
new_index.invlists.print_stats() # empty
new_index.ntotal # why 0?

<bound method IndexIVF.replace_invlists of <faiss.swigfaiss_avx2.IndexIVFFlat; proxy of <Swig Object of type 'faiss::IndexIVFFlat *' at 0x7fba9ca5de60> >>

<faiss.swigfaiss_avx2.VStackInvertedLists; proxy of <Swig Object of type 'faiss::VStackInvertedLists *' at 0x7fba9d355920> >

False

0

True

1001

1001

list size in < 1: 1 instances
list size in < 32: 3 instances
list size in < 64: 8 instances
list size in < 128: 32 instances
list size in < 256: 100 instances
list size in < 512: 211 instances
list size in < 1024: 288 instances
list size in < 2048: 255 instances
list size in < 4096: 90 instances
list size in < 8192: 13 instances


0

In [35]:
big_il.compute_ntotal()

1000000

In [37]:
ntotal = big_il.compute_ntotal()
new_index.ntotal = ntotal
new_index.ntotal

1000000

In [38]:
new_index

<faiss.swigfaiss_avx2.IndexIVFFlat; proxy of <Swig Object of type 'faiss::IndexIVFFlat *' at 0x7fba9ca5de60> >

In [40]:
xq[1:2,:]

array([[ 40.,  25.,  11.,   0.,  22.,  31.,   6.,   8.,  10.,   3.,   0.,
          1.,  30.,  91.,  88.,  18.,  38.,  44.,  16.,   1.,   5.,  25.,
         70.,  36.,   1.,  22.,  10.,   7.,  10.,  40.,  61.,   1.,  60.,
          9.,   8.,   4., 111.,  45.,  21.,  45.,   9.,   1.,   1.,  14.,
        111., 111.,  70.,  26.,  95.,  13.,   3.,   2.,   3.,  39., 111.,
        111.,  20.,   3.,  11.,  11.,   1.,  32.,  70.,  22.,  48.,   8.,
          9.,  25.,  60.,  26.,  14.,  37.,   4.,   5.,  65., 110., 111.,
         31.,   1.,   0., 101.,  78.,  84.,  34.,   4.,   2.,   2.,  29.,
         33.,  44.,  25.,  22.,   2.,   0.,   4.,  18.,  54.,  51.,  24.,
         21.,  12.,  18.,   5.,   6.,  11.,  17., 100.,  65.,  50.,  92.,
         37.,  14.,  23.,  77.,  95.,   9.,   3.,  14.,  60.,  40.,   4.,
         30.,  23.,  32.,  10.,   3.,  19.,  13.]], dtype=float32)

In [49]:
new_index.nprobe = 2
qid = 1
a, b = new_index.search(xq[qid:qid+1,:], k=100)
gt[qid:qid+1,:]
compute_recall(b, gt[qid:qid+1,:], k=100)

array([[413247, 413071, 706838, 880592, 249062, 400194, 942339, 880462,
        987636, 941776, 420802, 586780, 833054, 248426, 849742, 208391,
        271353, 192537, 987215, 832927,  20845, 948375, 266890, 202612,
        980641, 183334, 517923, 976547, 517238, 133524, 833335, 974985,
        560219, 731804,  71819, 980027, 538702, 766416, 987696, 122309,
        541805, 480949, 826670, 974930, 522078,  14836, 543219, 239335,
        250786, 833086, 406020, 598353, 491737, 957556, 956542, 214501,
        133522, 974481, 250884, 395932, 123544, 207435, 522482, 913887,
        807790, 808402, 850854, 509041, 378239, 411149, 868337, 837940,
        704973, 196686, 275502, 915764,  17970, 678724, 908803, 855295,
        741996, 913942, 490751, 739835, 730127, 618740, 394982, 253083,
        993635,   2781, 496660, 947005, 502126, 888719, 705714, 125095,
        862007, 855176, 846198, 987074]], dtype=int32)

0.28

In [51]:
index_to_crack

<faiss.swigfaiss_avx2.IndexIVFFlat; proxy of <Swig Object of type 'faiss::IndexIVFFlat *' at 0x7fba9d354180> >

In [57]:
index_to_crack.nprobe = 2
qid = 1
a, b = index_to_crack.search(xq[qid:qid+1,:], k=100)
gt[qid:qid+1,:]
compute_recall(b, gt[qid:qid+1,:], k=100)

array([[413247, 413071, 706838, 880592, 249062, 400194, 942339, 880462,
        987636, 941776, 420802, 586780, 833054, 248426, 849742, 208391,
        271353, 192537, 987215, 832927,  20845, 948375, 266890, 202612,
        980641, 183334, 517923, 976547, 517238, 133524, 833335, 974985,
        560219, 731804,  71819, 980027, 538702, 766416, 987696, 122309,
        541805, 480949, 826670, 974930, 522078,  14836, 543219, 239335,
        250786, 833086, 406020, 598353, 491737, 957556, 956542, 214501,
        133522, 974481, 250884, 395932, 123544, 207435, 522482, 913887,
        807790, 808402, 850854, 509041, 378239, 411149, 868337, 837940,
        704973, 196686, 275502, 915764,  17970, 678724, 908803, 855295,
        741996, 913942, 490751, 739835, 730127, 618740, 394982, 253083,
        993635,   2781, 496660, 947005, 502126, 888719, 705714, 125095,
        862007, 855176, 846198, 987074]], dtype=int32)

0.28

We see above how to add a new centroid to invlist... But points were not re-assigned. So you need to remove_ids () or resize.   
TBD adding this to cracking code, but this is how I can increase Q-means. I can seperate "addings" centroids with re-assigning them.
There is a remove_ids() implementation that is more efficient than what I hade with external IDMAP() object. I should look into it.   
There is also just .resize(0) and add preassigned all poitns again (which is what I am doing.)    
But if there is not enough re-assignments, only few points change clusters, then we can use the first more optimized version    


TODO:
- make sure previous indexes and created objects are deleted to avoid memory leak

In [34]:
new_index.ntotal

0

In [ ]:
quantizer = faiss.IndexFlatL2(d)
if metric == "euclidean":
    new_index = faiss.IndexIVFFlat(quantizer, d, new_nlist, faiss.METRIC_L2)
elif metric == "angular":
    new_index = faiss.IndexIVFFlat(quantizer, d, new_nlist, faiss.METRIC_INNER_PRODUCT)

new_index.cp.seed = 42
new_index.cp.niter = 0
new_index.cp.max_points_per_centroid = 256
new_index.cp.nredo = 1
new_index.verbose = True

strain = time.perf_counter()
index.train(data)
etrain = time.perf_counter()

sadd = time.perf_counter()
index.add(data)
eadd = time.perf_counter()

if store_dir is not None:
    filename = (
        store_dir
        + f"/index-n_iter_{km_n_iter}-nlist_{nlist}-max_pts_{km_max_pts}-seed_{seed}.index"
    )
    if args.store:
        print("storing index", filename)
        faiss.write_index(index, filename)

print(f"\t---> Index Train Time = {(etrain - strain)*1000} ms | Add Time = {(eadd - sadd)*1000} ms <---")
return index, etrain - strain, eadd - sadd

Check from the following how to do above (maybe it's here)

In [70]:
#!/usr/bin/env python3
# Copyright (c) Meta Platforms, Inc. and affiliates.
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.

import os
import faiss
import numpy as np


class CombinedIndex:
    """
    combines a set of inverted lists into a hstack
    masks part of those lists
    adds these inverted lists to an empty index that contains
    the info on how to perform searches
    """

    def __init__(self, invlist_fnames, empty_index_fname,
                 masked_index_fname=None):

        self.indexes = indexes = []
        ilv = faiss.InvertedListsPtrVector()

        for fname in invlist_fnames:
            if os.path.exists(fname):
                print('reading', fname, end='\r', flush=True)
                index = faiss.read_index(fname)
                indexes.append(index)
                il = faiss.extract_index_ivf(index).invlists
            else:
                raise AssertionError
            ilv.push_back(il)
        print()

        self.big_il = faiss.VStackInvertedLists(ilv.size(), ilv.data())
        if masked_index_fname:
            self.big_il_base = self.big_il
            print('loading', masked_index_fname)
            self.masked_index = faiss.read_index(
                masked_index_fname,
                faiss.IO_FLAG_MMAP | faiss.IO_FLAG_READ_ONLY)
            self.big_il = faiss.MaskedInvertedLists(
                faiss.extract_index_ivf(self.masked_index).invlists,
                self.big_il_base)

        print('loading empty index', empty_index_fname)
        self.index = faiss.read_index(empty_index_fname)
        ntotal = self.big_il.compute_ntotal()

        print('replace invlists')
        index_ivf = faiss.extract_index_ivf(self.index) # create a new empty index
        index_ivf.replace_invlists(self.big_il, False) # replace the invlists
        index_ivf.ntotal = self.index.ntotal = ntotal # the new ntotal
        index_ivf.parallel_mode = 1   # seems reasonable to do this all the time << CHEF: not sure what this is

        quantizer = faiss.downcast_index(index_ivf.quantizer)
        quantizer.hnsw.efSearch = 1024 # chef: also not sure what this is

    ############################################################
    # Expose fields and functions of the index as methods so that they
    # can be called by RPC

    def search(self, x, k):
        return self.index.search(x, k)

    def range_search(self, x, radius):
        return self.index.range_search(x, radius)

    def transform_and_assign(self, xq):
        index = self.index

        if isinstance(index, faiss.IndexPreTransform):
            assert index.chain.size() == 1
            vt = index.chain.at(0)
            xq = vt.apply_py(xq)

        # perform quantization
        index_ivf = faiss.extract_index_ivf(index)
        quantizer = index_ivf.quantizer
        coarse_dis, list_nos = quantizer.search(xq, index_ivf.nprobe)
        return xq, list_nos, coarse_dis


    def ivf_search_preassigned(self, xq, list_nos, coarse_dis, k):
        index_ivf = faiss.extract_index_ivf(self.index)
        n, d = xq.shape
        assert d == index_ivf.d
        n2, d2 = list_nos.shape
        assert list_nos.shape == coarse_dis.shape
        assert n2 == n
        assert d2 == index_ivf.nprobe
        D = np.empty((n, k), dtype='float32')
        I = np.empty((n, k), dtype='int64')
        index_ivf.search_preassigned(
            n, faiss.swig_ptr(xq), k,
            faiss.swig_ptr(list_nos), faiss.swig_ptr(coarse_dis),
            faiss.swig_ptr(D), faiss.swig_ptr(I), False)
        return D, I


    def ivf_range_search_preassigned(self, xq, list_nos, coarse_dis, radius):
        index_ivf = faiss.extract_index_ivf(self.index)
        n, d = xq.shape
        assert d == index_ivf.d
        n2, d2 = list_nos.shape
        assert list_nos.shape == coarse_dis.shape
        assert n2 == n
        assert d2 == index_ivf.nprobe
        res = faiss.RangeSearchResult(n)

        index_ivf.range_search_preassigned(
            n, faiss.swig_ptr(xq), radius,
            faiss.swig_ptr(list_nos), faiss.swig_ptr(coarse_dis),
            res)

        lims = faiss.rev_swig_ptr(res.lims, n + 1).copy()
        nd = int(lims[-1])
        D = faiss.rev_swig_ptr(res.distances, nd).copy()
        I = faiss.rev_swig_ptr(res.labels, nd).copy()
        return lims, D, I

    def set_nprobe(self, nprobe):
        # CHEF: extract_index_ivf : get an IndexIVF from an index. The index may be an IndexIVF or some wrapper class that encloses an IndexIVF
        index_ivf = faiss.extract_index_ivf(self.index)
        index_ivf.nprobe = nprobe

    def set_parallel_mode(self, pm):
        index_ivf = faiss.extract_index_ivf(self.index)
        index_ivf.parallel_mode = pm

    def get_ntotal(self):
        return self.index.ntotal

    def set_prefetch_nthread(self, nt):
        for idx in self.indexes:
            il = faiss.downcast_InvertedLists(
                faiss.extract_index_ivf(idx).invlists)
            il.prefetch_nthread
            il.prefetch_nthread = nt

    def set_omp_num_threads(self, nt):
        faiss.omp_set_num_threads(nt)

class CombinedIndexDeep1B(CombinedIndex):
    """ loads a CombinedIndex with the data from the big photodna index """

    def __init__(self):
        # set some paths
        workdir = "/checkpoint/matthijs/ondisk_distributed/"

        # empty index with the proper quantizer
        indexfname = workdir + 'trained.faissindex'

        # index that has some invlists that override the big one
        masked_index_fname = None
        invlist_fnames = [
            '%s/hslices/slice%d.faissindex' % (workdir, i)
            for i in range(50)
        ]
        CombinedIndex.__init__(self, invlist_fnames, indexfname, masked_index_fname)


def ivecs_read(fname):
    a = np.fromfile(fname, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy()


def fvecs_read(fname):
    return ivecs_read(fname).view('float32')


if __name__ == '__main__':
    import time
    ci = CombinedIndexDeep1B()
    print('loaded index of size ', ci.index.ntotal)

    deep1bdir = "/datasets01_101/simsearch/041218/deep1b/"

    xq = fvecs_read(deep1bdir + "deep1B_queries.fvecs")
    gt_fname = deep1bdir + "deep1B_groundtruth.ivecs"
    gt = ivecs_read(gt_fname)

    for nprobe in 1, 10, 100, 1000:
        ci.set_nprobe(nprobe)
        t0 = time.time()
        D, I = ci.search(xq, 100)
        t1 = time.time()
        print('nprobe=%d 1-recall@1=%.4f t=%.2fs' % (
            nprobe, (I[:, 0] == gt[:, 0]).sum() / len(xq),
            t1 - t0
        ))


AssertionError: 